In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
df_downloaded.head()

In [ ]:
df_downloaded.shape

In [ ]:
df_downloaded['message'][2695]

In [ ]:
len(df_downloaded[df_downloaded.message == 'spp-vsnap.cristie.se (Vsnap): free disk space 266,815 MB (16.32% free) lower than threshold 20%.'])

In [ ]:
# df[df['A'].str.contains("hello")]
#len(df_downloaded[df_downloaded['message'].str.contains('lower than threshold')]) => 600

pd.set_option('display.max_colwidth', -1)

df_lower=df_downloaded[df_downloaded['message'].str.contains('lower than threshold')]
df_lower['message']

In [ ]:
df_downloaded['message'][44]

In [ ]:
df_downloaded['message'][130]

In [ ]:
df_downloaded['message']

In [ ]:
df_downloaded

In [ ]:
df_downloaded['initMessageParams']

In [ ]:
sublist = df_downloaded['initMessageParams'][0]

In [ ]:
sublist

In [ ]:
print(sublist[0])

In [ ]:
df_downloaded['categoryDisplayName'].unique()

In [ ]:
#iter_df = df_downloaded['initMessageParams']
#df_downloaded['_unitLevel'] = ""
#for unitlevellist in iter_df:
#    #print(unitlevellist[0])
#    #print(iter_df)
#    unitlevel = unitlevellist[0]
#    df_downloaded._unitLevel = unitlevel

#for index, row in iter_df.iteritems():
    #print(index, row[0])  # debug
    #print(df_downloaded.iloc[index]) # debug
    #df_downloaded.iloc[index]._unitLevel = row[0]
    #print(df_downloaded.iloc[index]._unitLevel)
    #print("****************************************************")  # debug

In [ ]:
# df.assign(temp_f=lambda x: x.temp_c * 9 / 5 + 32)


# 'True' if x <= 4 else 'False'

#df_downloaded.assign(_unitLevel= 'Foo' if df_downloaded['initMessageParams'] == 'None' else df_downloaded['initMessageParams'][0])
df_downloaded['_unitLevel'] = df_downloaded['initMessageParams'].apply(lambda x: 'Foo' if x == 'None' else x[0])
#df['name_match'] = df['First_name'].apply(lambda x: 'Match' if x == 'Bill' else 'Mis-Match')




# OK, now we have an extra column called _unitLevel with the unit and level sorted out

In [ ]:
df_downloaded.head()

Save to object storage in new file: 

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
# feature engineering and select the important ones

df_reduced=df_downloaded_new[['acknowledged', 'alertTime', 'category', 'categoryDisplayName', 'count', 'dataSource','expired','expiresAt', 'first', 'initMessageParams','initialMessage','jobSessionId', 'last','message', 'messageName','messageParams','name','retention','status','statusDisplayName','storageId','type','typeDisplayName','unique', '_unitLevel']]
#print(df_reduced.head())
#print(df_reduced.shape)
df_reduced=df_reduced.dropna()
#df_reduced=df_reduced['_unitLevel'].drop_values('Foo','CertSrv1','Win2016Test', 'win2016-test','vmware_Templates','Report_1004_1000','Report_1003_1000','Report_1002_1000','Hypervisor Inventory', 'Deploy1', '_unitLevel','ubuntohog','Maintenance')
mask = df_reduced['_unitLevel'].isin(['Report_License','_unitLevel', 'Foo','CertSrv1','Win2016Test', 'win2016-test','vmware_Templates','Report_1004_1000','Report_1003_1000','Report_1002_1000','Hypervisor Inventory', 'Deploy1', '_unitLevel','ubuntuhog','Maintenance', 'Report_1005_1000','vmware_DLN_PROTECT01','vmware_DLN_TEST_DELETE_IN_JUNE2019',95.61194610595703,95.1344985961914, 95.78414916992188, 95.00507354736328,'Report_wtg','Report_1014_1000','Report_1015_1000','Report_1013_1000',95.17405700683594, 'Report_1012_1000','Report_1011_1000','Report_1001_1000',95.12422943115234, 95.47147369384766, 95.41291809082031,'onDemandRestore_1568284281377'])
df_reduced=df_reduced[~mask]

print(df_reduced.head())
print(df_reduced['_unitLevel'].unique())
#print(df_reduced.shape)
# save as csv in project for future use
#print(df_reduced.describe())
df_reduced.to_csv('./example_log_data.csv')
client_cred.upload_file('./example_log_data.csv', bucket, 'df_example_log.csv')

In [ ]:
df_downloaded_new.head(10)

In [ ]:
df_downloaded_new['_unitLevel']

In [ ]:
df_downloaded_new['_unitLevel'].unique()

In [ ]:
len(df_downloaded_new['_unitLevel'].unique())

In [ ]:
df_unitMessages=df_downloaded_new[['_unitLevel', 'messageName']]

df_sumOfMessagesPerUnit=pd.DataFrame(df_unitMessages.groupby(['_unitLevel', 'messageName']).size())
#df_unitMessages.groupby(['_unitLevel', 'messageName']).size()

#df.groupby('A').B.agg(['count']).rename(columns={'count': 'foo'})

df_test = df_downloaded_new.groupby(['_unitLevel', 'messageName']).agg(['count'])
df_test.head(25)

In [ ]:
# rename 0 column to sum
# df.rename({'a': 'X', 'b': 'Y'}, axis=1, inplace=True)
df_sumOfMessagesPerUnit.rename(columns={0: "SUM"}, inplace=True)

df_sumOfMessagesPerUnit

In [ ]:
#df.sort_values(by=['col1'])
df_sumOfMessagesPerUnit.sort_values(by=['SUM'], ascending=False)

In [ ]:
df_downloaded_new[df_downloaded_new['messageName'] == 'None'].message

In [ ]:
backup_msg=pd.DataFrame(df_downloaded_new[df_downloaded_new['messageName'] == 'None'].message)

In [ ]:
backup_msg.head()

In [ ]:
len(backup_msg['message'].unique())

In [ ]:
#len(backup_msg['message']) => 171
backup_msg

In [ ]:
backup_msg['message'].unique()

felet, frekv., antal, klustring => action

# Let's work on the timings and add a column for "last seen"

In [ ]:
# with cred to https://stackoverflow.com/questions/51856844/python-pandas-minutes-since-last-occurrence-in-2-million-row-dataframe

df_downloaded_new.alertTime = pd.to_datetime(df_downloaded_new.alertTime)
#df_downloaded_new.head(10)
df_downloaded_new['minDiff'] = (df_downloaded_new.alertTime -df_downloaded_new.alertTime.shift(1)).astype('timedelta64[m]')

In [ ]:
df_downloaded_new.head(10)

In [ ]:
the_mess1 = 'Log backup failed for database [WideWorldImporters] on instance [SQL2016]. An exception occurred while executing a Transact-SQL statement or batch.. Error: 0x80131501'
mask1 = (df_downloaded_new.message == the_mess1).cumsum().shift(1)

#print(mask1)

the_mess2 = 'Log backup failed for database [AdventureWorks2014] on instance [SQL2016]. An exception occurred while executing a Transact-SQL statement or batch.. Error: 0x80131501'
mask2 = (df_downloaded_new.message == the_mess2).cumsum().shift(1)
#print(mask2)
df_downloaded_new['minDiffMess1'] = df_downloaded_new.groupby(mask1.where(mask1 > 0)).minDiff.cumsum()
df_downloaded_new['minDiffMess2'] = df_downloaded_new.groupby(mask2.where(mask2 > 0)).minDiff.cumsum()

In [ ]:
#df_downloaded_new.minDiffMess = df_downloaded_new.minDiffMess.replace(0,method='ffill')

In [ ]:
df_downloaded_new.head(25)

In [ ]:
df_Mess=df_downloaded_new[df_downloaded_new['name'] == 'SQLLOG_ALERT_MESSAGE']
df_Mess=df_Mess[(df_Mess['message'] == the_mess1) | (df_Mess['message'] == the_mess2)]

In [ ]:
df_Mess[['alertTime','message','name','minDiffMess1','minDiffMess2']]

In [ ]:
from datetime import datetime

last_time=datetime(2019,1,1,0,0)
#print(last_time)
print(type(last_time))
def add_time(t):
    global last_time
    t_delta=t
    #print(type(t))
    t_delta= t-last_time
    #t_delta = (last_time-t).astype('timedelta64[m]')
    last_time = t
    print(t_delta)
    return t_delta

#df_downloaded_new['_lastSeen'] = df_downloaded_new['message'].apply(lambda x: add_time(df_downloaded_new['alertTime'].values) if x == the_mess1 else 0)
df_downloaded_new['_lastSeen'] = df_downloaded_new.apply(lambda row: add_time(row['alertTime']) if row['message'] == the_mess1 else 0, axis=1)


In [ ]:
#mess_array=df_downloaded_new['message'].unique()
#df_Foo=df_downloaded_new[df_downloaded_new['_unitLevel'] == 'Foo'] #vmware_Gold hyperv_Bronze catalog_Bronze sql_Bronze vmware_Bronze
df_Foo=df_downloaded_new[df_downloaded_new['_unitLevel'].isin(['Foo', 'vmware_Gold', 'hyperv_Bronze', 'catalog_Bronze', 'sql_Bronze', 'vmware_Bronze'])]
#print(df_Foo)
mess_array=df_Foo['_unitLevel'].unique()
#mess_array=['Log backup failed for database [WideWorldImporters] on instance [SQL2016]. An exception occurred while executing a Transact-SQL statement or batch.. Error: 0x80131501',
#            'Log backup failed for database [AdventureWorks2014] on instance [SQL2016]. An exception occurred while executing a Transact-SQL statement or batch.. Error: 0x80131501',
#            'Job vmware_Gold (id=1009, session=1,553,346,300,231) partially succeeded.']
print(len(mess_array))
mess_array = mess_array[:10]
mess_array

In [ ]:
from datetime import datetime
from datetime import timedelta


last_time=datetime(2019,3,23,13,5)
#print(last_time)
print(type(last_time))
def add_time(t):
    global last_time
    t_delta=t
    #print(type(t))
    t_delta= abs(t-last_time)
    #t_delta = (last_time-t).astype('timedelta64[m]')
    last_time = t
    #print(t_delta)
    #delta = timedelta(days=t_delta.days, hours=t_delta.hours, minutes=t_delta.minutes, seconds=t_delta.seconds)
    total_seconds = t_delta.total_seconds()
    minutes = int(total_seconds // 60)
    #print(type(minutes))
    t_delta=minutes
    return t_delta

def test_row(t,m,l):
    global messx
    if (m == messx):
        print('Found match!')
        return add_time(t)
    else :
        return l
    
for messx in mess_array:
    print(messx)
    last_time=datetime(2019,3,23,13,5)
    for label, row in df_downloaded_new.iterrows():
        print(label)
        df_downloaded_new.loc[label, '_lastSeen'] = test_row(row['alertTime'], row['_unitLevel'], row['_lastSeen'])
        print(df_downloaded_new.loc[label, '_lastSeen'])
        #df_downloaded_new['_lastSeen'] = df_downloaded_new.apply(lambda row: add_time(row['alertTime']) if row['message'] == mess else 0, axis=1)
    

In [ ]:
df_downloaded_new.head(50)

In [ ]:
# feature engineering and select the important ones

#df_reduced_2=df_downloaded_new[['acknowledged', 'alertTime', 'category', 'categoryDisplayName', 'count', 'dataSource','expired','expiresAt', 'first', 'initMessageParams','initialMessage','jobSessionId', 'last','message', 'messageName','messageParams','name','retention','status','statusDisplayName','storageId','type','typeDisplayName','unique', '_unitLevel', '_lastSeen']]
df_reduced_2=df_downloaded_new[['acknowledged', 'alertTime', 'category', 'categoryDisplayName', 'count', 'dataSource','expired','expiresAt', 'first', 'initMessageParams','initialMessage','jobSessionId', 'last','message', 'messageName','messageParams','name','retention','status','statusDisplayName','storageId','type','unique', '_unitLevel', '_lastSeen']]

#print(df_reduced.head())
#print(df_reduced.shape)
df_reduced_2=df_reduced_2.dropna()
#df_reduced=df_reduced['_unitLevel'].drop_values('Foo','CertSrv1','Win2016Test', 'win2016-test','vmware_Templates','Report_1004_1000','Report_1003_1000','Report_1002_1000','Hypervisor Inventory', 'Deploy1', '_unitLevel','ubuntohog','Maintenance')
#mask = df_reduced_2['_unitLevel'].isin(['Report_License','_unitLevel', 'Foo','CertSrv1','Win2016Test', 'win2016-test','vmware_Templates','Report_1004_1000','Report_1003_1000','Report_1002_1000','Hypervisor Inventory', 'Deploy1', '_unitLevel','ubuntuhog','Maintenance', 'Report_1005_1000','vmware_DLN_PROTECT01','vmware_DLN_TEST_DELETE_IN_JUNE2019',95.61194610595703,95.1344985961914, 95.78414916992188, 95.00507354736328,'Report_wtg','Report_1014_1000','Report_1015_1000','Report_1013_1000',95.17405700683594, 'Report_1012_1000','Report_1011_1000','Report_1001_1000',95.12422943115234, 95.47147369384766, 95.41291809082031,'onDemandRestore_1568284281377'])
mask = df_reduced_2['_unitLevel'].isin(['Report_License','_unitLevel','CertSrv1','Win2016Test', 'win2016-test','vmware_Templates','Report_1004_1000','Report_1003_1000','Report_1002_1000','Hypervisor Inventory', 'Deploy1', '_unitLevel','ubuntuhog','Maintenance', 'Report_1005_1000','vmware_DLN_PROTECT01','vmware_DLN_TEST_DELETE_IN_JUNE2019',95.61194610595703,95.1344985961914, 95.78414916992188, 95.00507354736328,'Report_wtg','Report_1014_1000','Report_1015_1000','Report_1013_1000',95.17405700683594, 'Report_1012_1000','Report_1011_1000','Report_1001_1000',95.12422943115234, 95.47147369384766, 95.41291809082031,'onDemandRestore_1568284281377'])

df_reduced_2=df_reduced_2[~mask]

print(df_reduced_2.head())
print(df_reduced_2['_unitLevel'].unique())
#print(df_reduced.shape)
# save as csv in project for future use
#print(df_reduced.describe())

df_reduced_2=df_reduced_2[df_reduced_2['name'] != 'SERVER_DOWN']
print(df_reduced_2['name'].unique())

df_reduced_2=df_reduced_2[df_reduced_2['name'] != 'VSNAP_STORAGE_SPACE_LOW']
print(df_reduced_2['name'].unique())


 
df_reduced_2.to_csv('./example_log_data_2.csv', index=False)
client_cred.upload_file('./example_log_data_2.csv', bucket, 'df_example_log_2.csv')

In [ ]:
df_reduced_2[df_reduced_2['name']== 'SERVER_DOWN'].count()